In [2]:
import pandas as pd
df=pd.read_csv("/content/product_reviews.csv")
df

,ProductID,UserID,Category,ReviewText,Rating,ReviewDate
0,P00001,U3292,Beauty,Front perform state science agent the. Imagine...,3,2023-07-23
1,P00002,U9426,Electronics,Many research upon whatever concern drive husb...,5,2024-12-24
2,P00003,U3416,Toys,Them increase city star thought. Indicate posi...,2,2025-04-16
3,P00004,U5130,Home,Still ahead single man decision. Some world fa...,4,2025-04-13
4,P00005,U4571,Electronics,People rock any look material marriage late. T...,1,2025-04-14
...,...,...,...,...,...,...
995,P00996,U8614,Sports,Sometimes relationship music scientist through...,5,2025-03-28
996,P00997,U5298,Beauty,Investment matter Mrs rock. Pressure a after s...,1,2025-03-29
997,P00998,U1310,Electronics,Would speak serious. Buy guess red others rest...,2,2023-09-03
998,P00999,U7630,Toys,Worker lay sense guy free food drug.,4,2025-02-17


In [3]:
df.shape

(1000, 6)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ProductID   1000 non-null   object
 1   UserID      1000 non-null   object
 2   Category    1000 non-null   object
 3   ReviewText  1000 non-null   object
 4   Rating      1000 non-null   int64 
 5   ReviewDate  1000 non-null   object
dtypes: int64(1), object(5)
memory usage: 47.0+ KB


In [5]:
texts=df['ReviewText'].astype(str).values
ratings=df['Rating'].astype(int).values

In [6]:
from tensorflow.keras.utils import to_categorical

In [7]:
rating_encoded=to_categorical(ratings-1)

In [8]:
rating_encoded

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.]])

In [37]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [10]:
from collections.abc import Sequence
tokenize=Tokenizer()
tokenize.fit_on_texts(texts)
Sequence=tokenize.texts_to_sequences(texts)

In [11]:
Sequence

[[666,
  481,
  108,
  299,
  109,
  11,
  110,
  572,
  393,
  394,
  482,
  483,
  484,
  300,
  485,
  395,
  301,
  396,
  746,
  573,
  747,
  302,
  43,
  32,
  486,
  667,
  213,
  214,
  668,
  574,
  215,
  12,
  487],
 [669,
  303,
  111,
  33,
  488,
  66,
  489,
  44,
  575,
  155,
  490,
  12,
  156,
  112,
  6,
  304,
  216,
  485,
  882,
  217,
  218,
  397,
  576,
  157,
  7,
  219,
  67,
  305,
  828,
  113,
  747,
  306],
 [398, 158, 399, 400, 114, 401, 883, 220],
 [748,
  115,
  20,
  829,
  221,
  491,
  884,
  828,
  402,
  222,
  21,
  403,
  68,
  159,
  670,
  749,
  160,
  161,
  223,
  404,
  162],
 [405,
  163,
  13,
  224,
  830,
  406,
  492,
  407,
  671,
  577,
  164,
  750,
  672,
  116,
  493,
  225],
 [304,
  578,
  831,
  226,
  967,
  408,
  115,
  165,
  115,
  117,
  579,
  227,
  832,
  307,
  308,
  924,
  673,
  228,
  833,
  674,
  494,
  22,
  495,
  309,
  69,
  961,
  580,
  496,
  32,
  310,
  581,
  70,
  311,
  582,
  409,
  675,
  305,
 

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
max_len=max(len(seq) for seq in Sequence)
X=pad_sequences(Sequence,maxlen=max_len)
y=rating_encoded

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [27]:
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense

In [28]:
from tensorflow.keras.models import Sequential

In [29]:
vocab_size=len(tokenize.word_index)+1
embedding_dim=64

In [30]:
model=Sequential([
    Embedding(input_dim=vocab_size,output_dim=embedding_dim,input_length=max_len),
    SimpleRNN(64),
    Dense(64,activation='relu'),
    Dense(y.shape[1],activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [31]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [32]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.2344 - loss: 1.6101 - val_accuracy: 0.1950 - val_loss: 1.6077
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5697 - loss: 1.4813 - val_accuracy: 0.2300 - val_loss: 1.6097
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7442 - loss: 1.2048 - val_accuracy: 0.2500 - val_loss: 1.6567
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8955 - loss: 0.6834 - val_accuracy: 0.2550 - val_loss: 1.8392
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9663 - loss: 0.3158 - val_accuracy: 0.2450 - val_loss: 2.1730
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9866 - loss: 0.1557 - val_accuracy: 0.2450 - val_loss: 2.4228
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9972 - loss: 0.0665 - val_accuracy: 0.2450 - val_loss: 2.5467
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0351 - val_accuracy: 0.2350 - v

In [33]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {accuracy:.2f}")


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2318 - loss: 2.9916 

Test Accuracy: 0.25


In [51]:
sample_tests=[
    "this phone is good",
    "this phone is bad"
]

In [52]:
sample_seq = tokenize.texts_to_sequences(sample_tests)
sample_pad = pad_sequences(sample_seq, maxlen=max_len)
predictions = model.predict(sample_pad)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step


In [56]:
import numpy as np

for text, pred in zip(sample_tests, predictions):
    predicted_rating = np.argmax(pred) + 1
    print(f"Review: {text}")
    print(f"Predicted rating: {predicted_rating}")

Review: this phone is good
Predicted rating: 2
Review: this phone is bad
Predicted rating: 2
